In [ ]:
codigo_canhero = 18027

In [ ]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

In [ ]:
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [ ]:
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_296354 where cast(split_part(canhero, \' / \', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_quema'] = datetime.strptime(data['fecha_quema'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_inspeccion'] = datetime.strptime(data['fecha_inspeccion'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

insp = collections.namedtuple("insp", data.keys())(*data.values())
print('end')

In [ ]:
insp

In [ ]:
query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_43414 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
fotos = amigocloud.get(project, query_fotos)['data']

#if fotos_filename['data'] != []:
#    dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
#    fotos.append(dic)

#fotos

In [ ]:
fotos

In [ ]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/01_template_operador.docx")

#descargar fotos y generar lista InlineImage

lista_fotos_inline = []
for foto in fotos:
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open('fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50))})

context = {'insp':insp, 'fotos':lista_fotos_inline}

doc.render(context)

# formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
cod_nom = insp.canhero.split(' / ')
file_name = cod_nom[0] + '_EC_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1]

doc.save('_' + file_name + '.docx')